In [ ]:
import sqlite3
import time
import json
from web3 import Web3

# RPC endpoint and contract configuration
RPC_URL = 'https://sepolia.gateway.tenderly.com'
CONTRACT_ADDRESS = '0x6e70b715531BC3aAbD5F26762973d8A1070c5bFD'
ABI = json.loads("""
[
  {"inputs":[],"stateMutability":"nonpayable","type":"constructor"},
  {"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"token","type":"address"},{"indexed":true,"internalType":"address","name":"recipient","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"AssetWithdrawn","type":"event"},
  {"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"address","name":"token","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"uint8","name":"orderType","type":"uint8"},{"indexed":false,"internalType":"uint256","name":"size","type":"uint256"},{"indexed":false,"internalType":"string","name":"side","type":"string"},{"indexed":false,"internalType":"string","name":"marketCode","type":"string"}],"name":"OrderPlaced","type":"event"},
  {"stateMutability":"payable","type":"fallback"},
  {"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
  {"inputs":[{"internalType":"uint8","name":"orderType","type":"uint8"},{"internalType":"uint256","name":"size","type":"uint256"},{"internalType":"string","name":"side","type":"string"},{"internalType":"string","name":"marketCode","type":"string"}],"name":"placeEthOrder","outputs":[],"stateMutability":"payable","type":"function"},
  {"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint8","name":"orderType","type":"uint8"},{"internalType":"uint256","name":"size","type":"uint256"},{"internalType":"string","name":"side","type":"string"},{"internalType":"string","name":"marketCode","type":"string"}],"name":"placeTokenOrder","outputs":[],"stateMutability":"nonpayable","type":"function"},
  {"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},
  {"stateMutability":"payable","type":"receive"}
]
""")

# Initialize Web3 instance
w3 = Web3(Web3.HTTPProvider(RPC_URL))

# Create contract instance
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)

# SQLite database configuration
DB_NAME = 'events.db'

def init_db():
    """Initializes the SQLite database and creates tables if they do not exist."""
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    
    # Create table for OrderPlaced events
    c.execute('''
        CREATE TABLE IF NOT EXISTS OrderPlaced (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            sender TEXT,
            token TEXT,
            amount TEXT,
            orderType INTEGER,
            size TEXT,
            side TEXT,
            marketCode TEXT,
            block_number INTEGER,
            tx_hash TEXT
        )
    ''')
    
    # Create table for AssetWithdrawn events
    c.execute('''
        CREATE TABLE IF NOT EXISTS AssetWithdrawn (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            token TEXT,
            recipient TEXT,
            amount TEXT,
            block_number INTEGER,
            tx_hash TEXT
        )
    ''')
    
    conn.commit()
    conn.close()

def save_order_placed(event):
    """Saves an OrderPlaced event to the SQLite database."""
    receipt = event
    sender = receipt['args']['sender']
    token = receipt['args']['token']
    amount = str(receipt['args']['amount'])
    orderType = receipt['args']['orderType']
    size = str(receipt['args']['size'])
    side = receipt['args']['side']
    marketCode = receipt['args']['marketCode']
    block_number = receipt['blockNumber']
    tx_hash = receipt['transactionHash'].hex()
    
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''
        INSERT INTO OrderPlaced (sender, token, amount, orderType, size, side, marketCode, block_number, tx_hash)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (sender, token, amount, orderType, size, side, marketCode, block_number, tx_hash))
    conn.commit()
    conn.close()
    print(f"Saved OrderPlaced event: tx_hash={tx_hash}")

def save_asset_withdrawn(event):
    """Saves an AssetWithdrawn event to the SQLite database."""
    receipt = event
    token = receipt['args']['token']
    recipient = receipt['args']['recipient']
    amount = str(receipt['args']['amount'])
    block_number = receipt['blockNumber']
    tx_hash = receipt['transactionHash'].hex()
    
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''
        INSERT INTO AssetWithdrawn (token, recipient, amount, block_number, tx_hash)
        VALUES (?, ?, ?, ?, ?)
    ''', (token, recipient, amount, block_number, tx_hash))
    conn.commit()
    conn.close()
    print(f"Saved AssetWithdrawn event: tx_hash={tx_hash}")

def main():
    init_db()
    print("Database initialized and ready.")

    # Create event filters starting from the latest block
    order_filter = contract.events.OrderPlaced.createFilter(fromBlock='latest')
    withdraw_filter = contract.events.AssetWithdrawn.createFilter(fromBlock='latest')
    
    print("Listening for events...")
    while True:
        try:
            # Poll for OrderPlaced events
            for event in order_filter.get_new_entries():
                save_order_placed(event)
            
            # Poll for AssetWithdrawn events
            for event in withdraw_filter.get_new_entries():
                save_asset_withdrawn(event)
            
            # Sleep briefly to avoid hammering the RPC endpoint
            time.sleep(2)
        except Exception as e:
            print(f"Error occurred: {e}")
            time.sleep(5)

if __name__ == '__main__':
    main()


Exception: Failed to connect to the Ethereum node

In [3]:
!pip install web3 

  Using cached web3-7.9.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached eth_abi-5.2.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached eth_account-0.13.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached eth_hash-0.7.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached eth_utils-5.2.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached hexbytes-1.3.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websockets-13.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached pyunormalize-16.0.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached yarl-1.18.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (69 kB)
  Using cached parsi